In [1]:
from main import init_config
from controller import Controller
from dataset.process import TokenParser

import numpy as np

ctrl = Controller(*init_config())

2021-12-27 00:55:14.487052: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Running the DreamProlog algorithm.
There was a problem with the provided arguments. The program will run in the default setting:
--configs prolog --logdir logdir
Namespace(act='elu', action_gnn_hidden_act=64, action_gnn_hidden_val=200, action_gnn_layers=6, action_gnn_next_shape=(32, 64, 32), action_gnn_start_shape=(4, 1, 4), action_repeat=1, actor_disc=5, actor_dist='normal', actor_entropy='1e-4', actor_grad_clip=100, actor_init_std=1.0, actor_layers=4, actor_lr=0.001, actor_min_std=0.1, actor_outscale=0.0, actor_state_entropy=0.0, actor_temp=0.1, atari_grayscale=False, behavior_stop_grad=True, clip_rewards='identity', collect_dyn_sample=True, dataset_size=100000, debug=False, disag_layers=1, disag_log=True, disag_models=3, disag_offset=0, disag_target='embed', disag_units=200, discount=0.95, discount_lambda=0.8, discount_layers=2, discount_scale=1.0, dyn_cell='gru', dyn_deter=512, dyn_discrete=32, dyn_hidden=512, dyn_input_layers=5, dyn_mean_act='none', dyn_min_std=0.1, dyn_output_lay

usage: ipykernel_launcher.py [-h] --configs CONFIGS [CONFIGS ...] --logdir
                             LOGDIR
ipykernel_launcher.py: error: the following arguments are required: --configs, --logdir


Logdir logdir


2021-12-27 00:55:15.942327: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-27 00:55:15.943052: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-27 00:55:17.237212: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-27 00:55:17.237887: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-27 00:55:17.237906: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-12-27 00:55:17.239219: I

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t30_wellord1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t40_tex_2.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Prefill dataset (0 steps).
Loaded problem: m2n140__t5_tmap_1
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t39_partfun1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Eval episode has 25 steps and return -0.500.


/home/kristsz/DreamProLog/envs/wrappers.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  value = np.array(value)


In [2]:
meta = ctrl.datasetManager._train_eps._meta
ds = []
for v in meta.values():
    ds.extend([str(s)[1:] for s in v['action_space_text']])
    
parser = TokenParser()

def pad(narr):
    size = narr.size
    return np.pad(narr, [0, 128-size])

parsed_ds = []
for dp in ds:
    parsed_ds.append(pad(np.array(parser.parse(dp), dtype = np.int32)))

In [3]:
import tensorflow as tf
BATCH_SIZE = 32
def generator():
    for i in range(1000):
        x = tf.constant([parsed_ds[np.random.randint(len(parsed_ds))] for i in range(BATCH_SIZE)])
        yield x, x
sgn = tf.TensorSpec((BATCH_SIZE, 128), dtype=tf.int32)
tf_ds = tf.data.Dataset.from_generator(generator, output_signature = (sgn, sgn))

2021-12-27 00:55:29.134409: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-27 00:55:29.353617: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-27 00:55:29.354239: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-27 00:55:29.354263: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] 

In [4]:
from unorganized.parts2 import Net

GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [5]:
class Loss(tf.keras.losses.Loss):
    def __init__(self, omega=0.01):
        super().__init__()
        self.omega = omega
        
    def call(self, y_true, y_pred):
        y = tf.nn.softmax(y_pred, axis=-1)
        y_probs = tf.gather(y, y_true, axis=-1, batch_dims=2)
        y_log_probs = tf.math.log(y_probs+0.001)
        #loss = self.omega*y_log_probs + (1-self.omega)*tf.math.cumsum(y_log_probs, axis=-1)
        #loss = y_log_probs*tf.math.cumprod(0.97*tf.ones((32,128)), axis=-1)
        mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
        #mask = tf.math.cumprod(mask, axis=-1)
        loss = (mask+self.omega)*y_log_probs
        loss = -tf.math.reduce_sum(loss)/(128*32)
        return loss

In [6]:
model = Net(4, 8, 128, 64, 4, 128)#Net(128, 4, 256)
#d_model = 256
#enc_embed = tf.keras.layers.Embedding(300, d_model)
#encoder = Encoder(4, d_model, 4, 512, 0.04)

In [7]:
x, x = next(iter(tf_ds))
model(x, False)
y_true = x
mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
mask = tf.math.cumprod(mask, axis=-1)

2021-12-27 00:55:30.340656: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-27 00:55:30.357404: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3598190000 Hz
2021-12-27 00:55:30.383214: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-12-27 00:55:30.542655: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=Loss(), metrics=[])


In [9]:
model.fit(tf_ds, epochs = 6)

Epoch 1/6
1000/1000 [==============================] - 24s 18ms/step - loss: 0.7817
Epoch 2/6
1000/1000 [==============================] - 18s 18ms/step - loss: 0.6761
Epoch 3/6
1000/1000 [==============================] - 20s 20ms/step - loss: 0.6668
Epoch 4/6
1000/1000 [==============================] - 18s 18ms/step - loss: 0.6669
Epoch 5/6
1000/1000 [==============================] - 18s 18ms/step - loss: 0.6680
Epoch 6/6
1000/1000 [==============================] - 18s 18ms/step - loss: 0.6689


In [10]:

x, y_true = next(iter(tf_ds))
y_pred = model(x, False)
y_pred = tf.nn.softmax(y_pred, axis=-1)
y_probs = tf.gather(y_pred, y_true, axis=-1, batch_dims=2)
y_pred = tf.math.argmax(y_pred, axis=-1)
print(x[:4], y_pred[:4], tf.cast(100*y_probs[:4], dtype=tf.int32), sep='\n\n')

tf.Tensor(
[[  3  21   5  10   7  22  23  24   5  38  23  19   7  38  23   6   7   2
    1   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [ 21   5  12  13   2   3  21   5  16  18   2   9   5  12  16   2   9   5
   13  18   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   

In [11]:
model.encoder.variable

<tf.Variable 'Variable:0' shape=(1, 8, 64) dtype=float32, numpy=
array([[[ 0.97824246,  0.5974416 ,  0.00375594,  0.23254953,
          0.22993088,  0.5955447 ,  0.17607917,  0.9732856 ,
          0.9335298 ,  0.40274316,  0.03229064,  0.3560708 ,
          0.6219572 ,  0.5183356 ,  1.0049845 ,  0.03811038,
          0.28507778,  0.42520964,  0.39790943,  0.8667803 ,
          0.6957693 ,  0.84650147,  0.11995757,  0.93961203,
          0.24121642,  0.37429434,  0.4334079 ,  0.7966627 ,
          0.0120051 ,  0.3501379 ,  0.04965518,  0.07324756,
          0.03087216,  0.40470082,  0.726252  ,  0.00433213,
          0.4536972 ,  0.85086304,  0.8707042 ,  0.15310346,
          0.23801196,  0.12061308,  0.25752205,  0.6786174 ,
          0.06412861,  0.71585965,  0.40815648,  0.42879924,
          0.7573862 ,  0.7762137 ,  0.92630833,  0.10193264,
          0.61462843,  0.3048497 ,  0.16122748,  0.6568268 ,
          0.4650954 ,  0.34682822,  0.08825079,  0.93316704,
          0.69303644

In [12]:
mask = tf.cast(tf.math.equal(x[:1], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
        
v, q = model.enc_embed(x)[:1], model.encoder.variable#tf.tile(model.encoder.variable, (32, 1, 1))
v = [v]
q = [q]
attention = []
for l in model.encoder.layers:
    _v, _q, _att = l(v[-1], q[-1], mask, False)
    v.append(_v)
    q.append(_q)
    attention.append(_att)
    

In [13]:
v

[<tf.Tensor: shape=(1, 128, 64), dtype=float32, numpy=
 array([[[-0.01382007, -0.04508341,  0.02874011, ..., -0.02775118,
          -0.02536368,  0.00064962],
         [ 0.01436266,  0.02622153,  0.01461639, ..., -0.01393499,
          -0.03265372,  0.00102618],
         [ 0.00857547,  0.0001025 ,  0.02118409, ..., -0.01031844,
          -0.02387962,  0.05068742],
         ...,
         [-0.03988689,  0.01468533,  0.04644993, ...,  0.05614553,
           0.02990331,  0.01761182],
         [-0.03988689,  0.01468533,  0.04644993, ...,  0.05614553,
           0.02990331,  0.01761182],
         [-0.03988689,  0.01468533,  0.04644993, ...,  0.05614553,
           0.02990331,  0.01761182]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 128, 64), dtype=float32, numpy=
 array([[[-0.20192027,  1.3405633 , -0.8428617 , ..., -0.2973917 ,
           1.1533742 , -1.2528737 ],
         [-0.20114128,  1.3411578 , -0.8436888 , ..., -0.29684174,
           1.153654  , -1.2516905 ],
         [-0.19997585,  1

In [14]:
attention[0]

(<tf.Tensor: shape=(1, 4, 128, 8), dtype=float32, numpy=
 array([[[[0.12545249, 0.12511896, 0.12447762, ..., 0.12503707,
           0.12401682, 0.12555017],
          [0.12500124, 0.12403779, 0.124701  , ..., 0.12725063,
           0.12449756, 0.1272593 ],
          [0.1250275 , 0.12406384, 0.12502721, ..., 0.12618999,
           0.12559298, 0.12666376],
          ...,
          [0.125     , 0.125     , 0.125     , ..., 0.125     ,
           0.125     , 0.125     ],
          [0.125     , 0.125     , 0.125     , ..., 0.125     ,
           0.125     , 0.125     ],
          [0.125     , 0.125     , 0.125     , ..., 0.125     ,
           0.125     , 0.125     ]],
 
         [[0.12381552, 0.1259679 , 0.1259109 , ..., 0.12662154,
           0.12292788, 0.12490745],
          [0.12466825, 0.12476002, 0.12428483, ..., 0.12738751,
           0.12321696, 0.12461508],
          [0.12265145, 0.12534714, 0.12570193, ..., 0.12721707,
           0.12363745, 0.12557124],
          ...,
          

In [15]:
q

[<tf.Variable 'Variable:0' shape=(1, 8, 64) dtype=float32, numpy=
 array([[[ 0.97824246,  0.5974416 ,  0.00375594,  0.23254953,
           0.22993088,  0.5955447 ,  0.17607917,  0.9732856 ,
           0.9335298 ,  0.40274316,  0.03229064,  0.3560708 ,
           0.6219572 ,  0.5183356 ,  1.0049845 ,  0.03811038,
           0.28507778,  0.42520964,  0.39790943,  0.8667803 ,
           0.6957693 ,  0.84650147,  0.11995757,  0.93961203,
           0.24121642,  0.37429434,  0.4334079 ,  0.7966627 ,
           0.0120051 ,  0.3501379 ,  0.04965518,  0.07324756,
           0.03087216,  0.40470082,  0.726252  ,  0.00433213,
           0.4536972 ,  0.85086304,  0.8707042 ,  0.15310346,
           0.23801196,  0.12061308,  0.25752205,  0.6786174 ,
           0.06412861,  0.71585965,  0.40815648,  0.42879924,
           0.7573862 ,  0.7762137 ,  0.92630833,  0.10193264,
           0.61462843,  0.3048497 ,  0.16122748,  0.6568268 ,
           0.4650954 ,  0.34682822,  0.08825079,  0.93316704,
    

In [16]:
tf.ones((32, 4, 128, 128))

<tf.Tensor: shape=(32, 4, 128, 128), dtype=float32, numpy=
array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],